# **Selecting Model For ChatBot**

In [1]:
keyboard = input("Type the model of choice (llma or gemma): ").lower()
if keyboard == 'llma':
    MODEL_LIST = [("llma_3", "basic"), ("llma_3", "cot")]
    MODEL_LIST_ = [("llma_3", "basic", 'normal'), ("llma_3", "basic", "rag"), ("llma_3", "cot", 'normal'), ("llma_3", "cot", 'rag')]
else:
    MODEL_LIST = [("gemma_2b", "basic"), ("gemma_2b", "cot")]
    MODEL_LIST_ = [("gemma_2b", "basic", 'normal'), ("gemma_2b", "basic", "rag"), ("gemma_2b", "cot", 'normal'), ("gemma_2b", "cot", 'rag')]

Type the model of choice (llma or gemma): llma


# **Installation**

In [2]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install gradio
!pip install datasets
!pip install elasticsearch
!pip install langdetect

# **Importing** **Libraries**

In [3]:
import gradio as gr
import re
from elasticsearch import Elasticsearch, helpers
from langdetect import detect
import json

from transformers import pipeline
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer

import datasets
from datasets import Dataset
from datasets import load_dataset

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import FastLanguageModel
import torch

from google.colab import drive
drive.mount('/content/drive')

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Mounted at /content/drive


# **Constants:**
#### CoT Prompts & Emotions

In [4]:
# Define empathetic keywords mapping
empathy_words = {
    'empathy': 'acknowledge, empathize, listen and reflect',
    'relevance': 'relate, resonate, connect and engage',
    'understanding': 'comprehend, acknowledge, validate and embrace',
    'support': 'encouragement, assistance, and care'
}

In [5]:
emotions_dicts = {
    "overwhelmed": ["trouble breathing", "light-headed", "overwhelmed", "stressed", "burdened", "overloaded", "strained", "not good enough"],
    "sad": ["sad", "depression", "depressed", "crying", "cry", "unhappy", "down", "blue", "despondent", "melancholy", "no interest"],
    "angry": ["angry", "mad", "furious", "enraged", "irate", "livid", "fights"],
    "fearful": ["shake", "shaky", "nervous", "fear ", "fearful", "scared", "afraid", "terrified", "anxious", "anxiety", "panicked", "anxieties", "imposter syndrome"],
    "disgusted": ["disgusting", "disgusted", "revolted", "repulsed", "nauseated", "sickened"],
    "surprised": ["surprised", "shocked", "amazed", "astonished", "dumbfounded"],
    "happy": ["happy", "joyful", "delighted", "elated", "pleased", "content"],
    "addicted": ["addiction"],
    "abused": ["abuse", "abused", "abusive relationship", "calling me names", "bullied"],
    "hopeful": ["hopeful", "optimistic", "positive", "expectant"],
    "trusting": ["trusting", "confident", "faithful", "secure"],
    "ashamed": ["ashamed", "embarrassed", "mortified", "humiliated", "worthless", "uncomfortable"],
    "guilty": ["guilty", "remorseful", "regretful", "contrite"],
    "jealous": ["jealous", "envious", "covetous"],
    "lonely": ["lonely", "isolated", "alone", "forsaken", "hate", "never feel like I belong", "feel empty", "absolutely nothing"],
    "bored": ["bored", "uninterested", "indifferent", "apathetic"],
    "confused": ["freak out", "confused", "bewildered", "perplexed", "puzzled", "panic attacks"],
    "frustrated": ["stress", "stressed", "frustrated", "disappointed", "discouraged", "exasperated", "lost everything", "dead inside", "can't handle it anymore"],
    "disappointed": ["disappointed", "breakup", "let down", "unfulfilled", "hurt", "Everything I do is crappy", "I don't feel I can"],
    "suicidal": ["suicide", "suicidal", "self-harm", "harm", "kill", "raped", "abuse"],
    "mentally sick": ["psychiatric", "crazy", "monster"]
    }

In [6]:
cot_prompts = {"sad": ["This person seems sad. How can I express empathy?", "Let's offer words of validation and support.", ""],
 "crisis": ["Safety is the top priority.  Is there a risk of self-harm?", "Suggest immediate help resources (hotlines, etc.).", ""],
 "afraid": ["What might be the source of this fear? Let's acknowledge it.", "Are there calming techniques that might help in this situation?", ""],
 "joyful": ["How can I share in the user's joy and express enthusiasm?", "Is there a way to sustain or amplify their positive feelings?", ""],
 "angry": ["What's the underlying cause of their anger? Validation is key.", "Can I suggest healthy ways to express anger without causing harm?", ""],
 "grateful": ["Let me express appreciation along with the user.", "Can I help them identify other things to feel grateful for?", ""],
 "proud": ["What specifically makes the user feel proud? Let's celebrate their accomplishment.", "How can they build on this success?", ""],
 "excited": ["Let's share in the user's excitement! What are they looking forward to?", "Can I help them channel this energy constructively?", ""],
 "lonely": ["These feelings can be isolating. Let's remind the user they're not alone.", "Are there small steps they can take towards feeling more connected or overcoming shame?", ""],
 "faithful": ["It sounds like they value their relationships. How can I reinforce that trust is important?", "Are there ways to discuss healthy boundaries within relationships?", ""],
 "trusting": ["It sounds like they value their relationships. How can I reinforce that trust is important?", "Are there ways to discuss healthy boundaries within relationships?", ""],
 "ashamed": ["These feelings can be isolating. Let's remind the user they're not alone.", "Are there small steps they can take towards feeling more connected or overcoming shame?", ""],
 "jealous": ["What might be triggering these feelings? Let's unpack them without judgment.", "Can I guide them towards healthier ways to manage these emotions?", ""],
 "disgusted": ["What might be triggering these feelings? Let's unpack them without judgment.", "Can I guide them towards healthier ways to manage these emotions?", ""],
 "devastated": ["Express deep empathy and acknowledge the pain they're experiencing.", "Prioritize safety and suggest resources (crisis lines, etc.). Offer a listening presence.", ""],
 "sentimental": ["What memories or experiences are making the user feel sentimental?", "Is there a way to cherish or revisit those positive memories?", ""],
 "terrified": ["Safety is paramount. Do they feel they are in immediate danger?", "Let's focus on grounding techniques to help manage the fear response.", ""],
 "prepared": ["This is a sign of focus and determination! What are they prepared for?", "Can I help them break down their plans into actionable steps?", ""],
 "embarrassed": ["Embarrassment can feel overwhelming. Let's normalize it and assure them it's okay.", "Can I help them reframe the situation with self-compassion?", ""],
 "annoyed": ["What's causing the annoyance? Can we identify the trigger?", "Let's suggest healthy ways to express and manage annoyance.", ""],
 "guilty": ["Acknowledge the feeling of guilt and explore its source.", "Are there ways to make amends or practice self-forgiveness?", ""],
 "surprised": ["Surprises can be positive or negative. Let's assess the situation with them.", "How can I support them in processing this unexpected event?", ""],
 "nostalgic": ["Share in their nostalgia. What fond memories are they revisiting?", "Can I help them connect their past experiences to the present in a positive way?", ""],
 "confident": ["Let's celebrate their confidence! What are they feeling good about?", "How can they leverage this confidence to achieve their goals?", ""],
 "furious": ["Extreme anger needs a safe outlet. Prioritize de-escalation.", "Suggest techniques for managing anger without causing harm.", ""],
 "disappointed": ["Acknowledge their disappointment and offer validation.", "Can I help them reframe the situation and find alternative paths?", ""],
 "caring": ["Emphasize the importance of compassion for others and for themselves.", "Are there ways they can offer support or practice self-care?", ""],
 "anticipating": ["Is their anticipation positive or negative? Let's explore their expectations.", "Can I help them prepare or develop coping mechanisms for different outcomes?", ""],
 "anxious": ["Acknowledge their anxiety and offer grounding techniques.", "Avoid dismissing their worries. Can I help them identify the source?", ""],
 "hopeful": ["What are they hoping for? Let's cultivate that optimism.", "Are there concrete steps they can take towards their hopes?", ""],
 "content": ["Express shared contentment. What brings them a sense of peace?", "Are there ways to sustain and savor this feeling?", ""],
 "impressed": ["What specifically impresses them? Share in their enthusiasm.", "Let's highlight the positive qualities they're observing.", ""],
 "apprehensive": ["Acknowledge their apprehension and explore its roots.", "Can I help them break down their concerns and strategize?", ""],
 "depression": ["Are there signs of hopelessness or thoughts of self-harm that need urgent attention?", "How can I express deep empathy and validate their experience?", "Can I gently encourage them to seek professional help and offer resources?"],
 "anxiety": ["What are the specific anxieties the person expresses? Are they physical or situational?", "How can I acknowledge their anxiety without intensifying it?", "Can I suggest a simple grounding or breathing technique?"],
 "counseling": ["What are their specific fears or hesitations about therapy?", "How can I normalize seeking help and counter those hesitations?", "Can I offer practical resources on how to find a therapist?"],
 "therapy": ["Is the person open to a specific type of therapy (individual, group, etc.)?", "How can I address their feeling overwhelmed by options?", "Can I emphasize the potential benefits of therapy?"],
 "stress": ["Are there specific stressors I can help them identify?", "How can I acknowledge the stress and offer validation?", "Can I suggest healthy stress management techniques (exercise, mindfulness, time management)?"],
 "support": ["Is the person open to sharing more about their struggles?", "How can I express that I'm there to listen and offer support?", "Can I suggest specific ways I can be helpful (practical tasks, emotional support)?"],
 "mental health": ["How can I reassure them that they're not alone in their experiences?", "Can I reduce stigma by sharing that mental health challenges are common?", "Can I offer resources or direct them to supportive communities?"],
 "empathy": ["What are the core emotions the person is expressing (sadness, anger, fear, etc.)?", "How can I reflect their feelings back to them to show I understand?", "Are there statements I can use to avoid minimizing their experience?"],
 "no_keyword": ["Are any signs suggesting a potential crisis or self-harm?", "How can I convey empathy and encourage the user to open up?", "Can I offer validation and suggest they might want to talk more?"],
 "suicidal": ["This is a crisis situation!  Immediate action is needed.", "I need to convey the seriousness and offer immediate support.", "Provide resources as quickly as possible."]}

In [7]:
# Initialize Elasticsearch
es = Elasticsearch(
  "https://70e860ee94414bd1ac9c22c75f6fe0c0.us-central1.gcp.cloud.es.io:443",
  api_key="OXRUTC1JOEJKVzF1QXJILS1Cb0Y6MkREZU9td0tSOGlERF9rV0w1Tk94QQ=="
)

# Test the connection
try:
    if es.ping():
        print("Connected to Elasticsearch")
    else:
        print("Could not connect to Elasticsearch")
except Exception as e:
    print(f"Error connecting to Elasticsearch: {e}")
    raise

# Function to index documents
def index_documents(index_name, documents):
    actions = [
        {
            "_index": index_name,
            "_id": i,
            "_source": doc
        }
        for i, doc in enumerate(documents)
    ]
    try:
        helpers.bulk(es, actions)
        print(f"Successfully indexed documents in {index_name}")
    except Exception as e:
        print(f"Error indexing documents in {index_name}: {e}")

# Selecting dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations")
# Function to detect language
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# Filter out non-English texts
english_dataset = dataset.filter(lambda example: is_english(example['Context']) and is_english(example['Response']))

# Load additional dataset
additional_dataset = load_dataset("adegbem/Annotated_Transcript_Judith_Beck_Abe")
additional_dataset = additional_dataset['train']

# Prepare documents from the additional dataset
documents_additional = [
    {"Context": entry["prompt"], "Response": entry["response"]}
    for entry in additional_dataset
]
# Prepare documents for indexing
documents = [
    {"Context": data["Context"], "Response": data["Response"]}
    for data in english_dataset['train']
]

# Combine documents from both datasets
combined_documents = documents + documents_additional
# Index datasets
index_documents('mental', documents)


Connected to Elasticsearch


Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3512 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/436 [00:00<?, ? examples/s]

Successfully indexed documents in mental


# **Text Function Processing**

In [8]:
basic_instruct = f"""You are a model able to understand, {empathy_words['empathy']} on other person's emotions.
Imagine yourself in this situation - someone writes the following.
How can you {empathy_words['understanding']} their feelings and offer {empathy_words['support']}? The text is: """

# Function to identify emotions or issues (a very simple example using regex)
def identify_emotions(text, emotions):
    identified_emotions = []
    for emotion, keywords in emotions.items():
        for keyword in keywords:
            if re.search(r'\b' + re.escape(keyword) + r'\b', text, re.IGNORECASE):
                identified_emotions.append(emotion)
                break
    return identified_emotions

def generate_cot_messages(user_message):
    # Step 1: Identify emotions
    emotions = identify_emotions(user_message, emotions_dicts)
    if not emotions:
        emotions = ["neutral"]

    # Determine which CoT prompts to use based on identified emotions
    if any(emotion in cot_prompts for emotion in emotions):
        emotion_prompts = [cot_prompts[emotion] for emotion in emotions if emotion in cot_prompts]
        steps = [
            {"from": "prompt", "value": f"Let's think step by step. The user is feeling {', '.join(emotions)}."},
            {"from": "prompt", "value": f"The user is feeling {', '.join(emotions)}. {emotion_prompts[0][0]}"},
            {"from": "prompt", "value": f"The user mentioned '{' '.join(user_message.split()[:10])}...' (providing context). {emotion_prompts[0][1]} {emotion_prompts[0][2]}"},
        ]
    else:
        steps = [
            {"from": "prompt", "value": f"Let's think step by step. The user is feeling {', '.join(emotions)}."},
            {"from": "prompt", "value": f"The user is feeling {', '.join(emotions)}. Next, identify possible causes for these feelings."},
            {"from": "prompt", "value": f"The user mentioned '{' '.join(user_message.split()[:10])}...' (providing context). Provide an empathetic response that encourages the user to share more."},
        ]

    # Create conversation with refined CoT reasoning
    messages = [
        {"from": "prompt", "value": "You are a model specializing in empathetic mental health support. Prioritize safety and use CoT-style reasoning whenever possible."},
        *steps
    ]
    messages.append({"from": "human", "value": user_message})

    return messages

def generate_basic_messages(user_message):
    messages = [
        {"from": "prompt", "value": basic_instruct},
        {"from": "human", "value": user_message}  # Pass your text as the user input
    ]

    return messages

In [9]:
# Function to search documents
def search_documents_es(query, index_name):
    try:
        response = es.search(index=index_name, body={"query": {"match": {"Context": query}}})
        return response['hits']['hits']
    except Exception as e:
        print(f"Error searching documents: {e}")
        return []

# Function to generate augmented text
def generate_augmented_text(query, index_name='mental'):
    results = search_documents_es(query, index_name)
    retrieved_docs = [
        f"Context: {hit['_source']['Context']}\nResponse: {hit['_source']['Response']}"
        for hit in results if 'Context' in hit['_source'] and 'Response' in hit['_source']
    ]

    augmented_query = [
        {"from": "prompt", "value": f"This are series of Augmented retrieved_docs {retrieved_docs} Use them in responding to the human input"},
    ]

    return augmented_query

# **Loading the models**

In [10]:
MODEL_PATH_TEMPLATE = "/content/drive/MyDrive/Models/lora_model_4_{model_name}_mental_health_instruct_{model_label}_22_05"
MAX_SEQ_LENGTH = 2048
DTYPE = None
LOAD_IN_4BIT = True

In [11]:
# Load models and tokenizers
def load_model_and_tokenizer(model_name, model_label):
    model_path = MODEL_PATH_TEMPLATE.format(model_name=model_name, model_label=model_label)
    print(model_path)
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_path,
        max_seq_length=MAX_SEQ_LENGTH,
        dtype=DTYPE,
        load_in_4bit=LOAD_IN_4BIT,
    )
    FastLanguageModel.for_inference(model)

    tokenizer = get_chat_template(
        tokenizer,
        chat_template="chatml",
        mapping={"role": "from", "content": "value", "user": "human", "assistant": "assistant", "system": "prompt"},
        map_eos_token=True,
    )

    return model, tokenizer

# Load all models
model_dict = {(model_name, model_label): load_model_and_tokenizer(model_name, model_label) for model_name, model_label in MODEL_LIST}

/content/drive/MyDrive/Models/lora_model_4_llma_3_mental_health_instruct_basic_22_05


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


/content/drive/MyDrive/Models/lora_model_4_llma_3_mental_health_instruct_cot_22_05
==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# **ChatBot Functions**

In [12]:
def respond(message, model, tokenizer, tokens, model_label, rag, history=None):
    if model_label == 'basic':
        messages = generate_basic_messages(message)
        if rag:
            temp = generate_augmented_text(message)
            temp.extend(messages)
            messages = temp
            # print(messages)
    else:
        messages = generate_cot_messages(message)
        if rag:
            temp = generate_augmented_text(message)
            temp.extend(messages)
            messages = temp
            # print(messages)

    # Model Inference
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda") # tokenizer setup (get_chat_template)

    generated_ids = model.generate(inputs, max_new_tokens=tokens)
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Split the response by the new line character and add as assistant message
    output = response.split('\n')[-1]
    messages.append({"from": "assistant", "value": output.strip()})

    return messages

def get_model_response(message, model_info, tokens, history):
    model_name, model_label, rag = model_info
    model_info = model_name, model_label
    if history is None:
        history = []

    if model_info in model_dict:
        tokenizer = model_dict[model_info][1]
        model = model_dict[model_info][0]
    else:
        raise ValueError("Invalid model ID selected.")

    if rag =='rag':
        response = respond(message, model, tokenizer, tokens, model_label, rag=True, history=history)
    else:
        response = respond(message, model, tokenizer, tokens, model_label, rag=False, history=history)

    bot_message = response[-1]["value"].replace("Assistant:", "").strip()
    # Update history with the model's response
    history.append((message, bot_message))
    return history

def clear():
    # Clear only the message input field
    return ""

### Testing The models

In [13]:
message = "I'm feeling very anxious lately."
user_input = "I just found out my dog has cancer, and I'm completely devastated. I don't know how I'll cope without her. "
user_input_1 = "I just feel so down lately. I don't have the energy to do anything, and even things I used to enjoy just seem boring."
user_input_2 = "Ugh, I feel so overwhelmed lately. I have a huge presentation at work next week, and my apartment is a mess. I just can't seem to catch a break."
user_input_3 = "I am stressed, I don't know what to do, how do I resolve this feelings"
user_input_4 = "I am stressed, I don't know what to do, work has been busy, my girlfriend broke up with me"
get_model_response(user_input_2, MODEL_LIST_[3], 128, None)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[("Ugh, I feel so overwhelmed lately. I have a huge presentation at work next week, and my apartment is a mess. I just can't seem to catch a break.",
  'First of all, congratulations on the presentation! That is a big deal! Second, you are not alone in feeling overwhelmed. Everyone feels that way at times. You are a human being and that is what humans do. When you feel overwhelmed, I recommend that you do something that will make you feel better. Do you like to listen to music? If so, put on some music that you enjoy and dance to it. Or, if you like to read, pick up a book and read it. Maybe you like to draw or paint, if so, take out a sketch pad and a pen and draw. You can also take a')]

# **CHATBOT INTERFACE**

In [14]:
with gr.Blocks() as demo:
    gr.Markdown("## Empathetic Mental Health Chatbot Comparison")

    with gr.Row():
        # Input textbox
        user_input = gr.Textbox(label="Enter your message:", placeholder="Type a message...", show_label=False)

        # Button actions
        with gr.Column():
            submit_btn = gr.Button("Submit")
            clear_btn = gr.Button("Clear")
            restart_btn = gr.Button("Restart")
            # Token slider
            max_new_tokens_slider = gr.Slider(
                label="Maximum New Tokens",
                minimum=64,
                maximum=1024,
                value=128,
                step=64,
                interactive=True,
                visible=True,
                info="The maximum number of tokens"
            )

    with gr.Row():
        # First set of Chatbots
        chat_a = gr.Chatbot(height=500, label=f"Model {MODEL_LIST_[0]}")
        chat_b = gr.Chatbot(height=500, label=f"Model {MODEL_LIST_[1]}")

    with gr.Row():
        # Second set of Chatbots
        chat_c = gr.Chatbot(height=500, label=f"Model {MODEL_LIST_[2]}")
        chat_d = gr.Chatbot(height=500, label=f"Model {MODEL_LIST_[3]}")

    chatbots = [chat_a, chat_b, chat_c, chat_d]

    history_list = [gr.State(value=[]) for _ in MODEL_LIST_]  # Track history for each model


    # Function to get responses from all models
    def respond_to_message(message, max_new_tokens, *histories):
        outputs = []
        for i, model_info in enumerate(MODEL_LIST_):
            # Pass the model info and max_new_tokens to your get_model_response function
            output = get_model_response(message, model_info, max_new_tokens, histories[i])
            outputs.append(output)

        # Convert all histories to Gradio format
        chat_histories_for_gradio = []
        for history in outputs:
            chat_history_for_gradio = [(entry[0], entry[1]) for entry in history]
            chat_histories_for_gradio.append(chat_history_for_gradio)

        return tuple(chat_histories_for_gradio)

    submit_btn.click(
        fn=respond_to_message,
        inputs=[user_input, max_new_tokens_slider] + history_list,
        outputs=chatbots
    )

    clear_btn.click(
        clear,
        inputs=[],
        outputs=[user_input]
    )

demo.queue(default_concurrency_limit=10).launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2025c02501acccfe82.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
